<a href="https://colab.research.google.com/github/kundajelab/label_shift_experiments/blob/master/diabetic_retinopathy/DiabeticRetinopathyDomainAdaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://zenodo.org/record/3412990/files/kaggle_dr.txt.gz?download=1 -O kaggle_dr.txt.gz
!wget https://zenodo.org/record/3412990/files/messidor_dr.txt.gz?download=1 -O messidor_dr.txt.gz

--2019-09-16 20:14:44--  https://zenodo.org/record/3412990/files/kaggle_dr.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106643 (104K) [application/octet-stream]
Saving to: ‘kaggle_dr.txt.gz’

kaggle_dr.txt.gz    100%[===================>] 104.14K   315KB/s    in 0.3s    

2019-09-16 20:14:46 (315 KB/s) - ‘kaggle_dr.txt.gz’ saved [106643/106643]

--2019-09-16 20:14:46--  https://zenodo.org/record/3412990/files/messidor_dr.txt.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.65.20
Connecting to zenodo.org (zenodo.org)|188.184.65.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38723 (38K) [application/octet-stream]
Saving to: ‘messidor_dr.txt.gz’

messidor_dr.txt.gz  100%[===================>]  37.82K  --.-KB/s    in 0.1s    

2019-09-16 20:14:47 (344 KB/s) - ‘messidor_dr.txt.gz’ saved [38723/38723]



In [0]:
import gzip
import numpy as np

kaggle_onehot_labels = [] #5 classes
kaggle_predlogits = []
for row in gzip.open("kaggle_dr.txt.gz"):
    _, kaggle_label, logits = row.decode("utf-8").rstrip().split("\t")
    kaggle_label = int(kaggle_label)
    logits = [float(x) for x in logits.split(",")]
    labelsvec = np.zeros(5)
    labelsvec[kaggle_label] = 1.0
    kaggle_onehot_labels.append(labelsvec)
    kaggle_predlogits.append(logits)
kaggle_onehot_labels = np.array(kaggle_onehot_labels)
kaggle_predlogits = np.array(kaggle_predlogits)

messidor_onehot_labels = [] #4 classes
messidor_predlogits = []
for row in gzip.open("messidor_dr.txt.gz"):
    _, messidor_label, logits = row.decode("utf-8").rstrip().split("\t")
    messidor_label = int(messidor_label)
    logits = [float(x) for x in logits.split(",")]
    labelsvec = np.zeros(4)
    labelsvec[messidor_label] = 1.0
    messidor_onehot_labels.append(labelsvec)
    messidor_predlogits.append(logits)
messidor_onehot_labels = np.array(messidor_onehot_labels)
messidor_predlogits = np.array(messidor_predlogits)

In [3]:
from sklearn.metrics import roc_auc_score
from scipy.special import softmax

print(roc_auc_score(y_true=1-kaggle_onehot_labels[:,0],
                    y_score=1-softmax(kaggle_predlogits, axis=-1)[:,0]))
print(roc_auc_score(y_true=1-messidor_onehot_labels[:,0],
                    y_score=1-softmax(messidor_predlogits, axis=-1)[:,0]))

0.8945151902722942
0.8920633498604387


In [4]:
![[ -e abstention ]] || git clone https://github.com/blindauth/abstention
%cd /content/abstention
!git checkout iterativebias
!git pull
!pip uninstall abstention
!pip install .
%cd ..

Cloning into 'abstention'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 278 (delta 39), reused 38 (delta 21), pack-reused 222
Receiving objects: 100% (278/278), 86.26 KiB | 5.75 MiB/s, done.
Resolving deltas: 100% (181/181), done.
/content/abstention
Branch 'iterativebias' set up to track remote branch 'iterativebias' from 'origin'.
Switched to a new branch 'iterativebias'
Already up to date.
Processing /content/abstention
  Created wheel for abstention: filename=abstention-0.1.2.1-cp36-none-any.whl size=45664 sha256=2ec92924d52d98f7b6b92dba046263f15d6c3686687555ea3be90850dc594e83
  Stored in directory: /tmp/pip-ephem-wheel-cache-dbqwk986/wheels/60/8e/d2/9e9ca02e7b5f76bfda2e2daa6dcbe42c19095c502ccb653729
Successfully built abstention
/content


In [0]:
#do calibration and domain adaptation
from collections import OrderedDict
from importlib import reload
import abstention
reload(abstention)
reload(abstention.calibration)
reload(abstention.label_shift)
from abstention.calibration import (
    TempScaling, CrossValidatedBCTS, ConfusionMatrix, softmax)
from abstention.label_shift import (EMImbalanceAdapter, BBSEImbalanceAdapter)

calibname_to_calibfactory = OrderedDict([
    ('None', abstention.calibration.Softmax()),
    ('TS', TempScaling(verbose=False)),
    ('BCTS', TempScaling(verbose=False,
                         bias_positions=[0,1,2,3,4])),
    #('CVBCTS', CrossValidatedBCTS(verbose=True,
    #                              num_crossvalidation_splits=5,
    #                              betas_to_try=[0.0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0, 10]
    #                             ))
])

adaptname_to_adapter = OrderedDict([
    ('em', EMImbalanceAdapter()),
    ('bbse-hard', BBSEImbalanceAdapter(soft=False)),
    ('bbse-soft', BBSEImbalanceAdapter(soft=True))
])

calibname_to_calibfunc = OrderedDict()
calibname_to_calibkaggle = OrderedDict()
calibname_to_calibmessidor = OrderedDict()
for calibname in calibname_to_calibfactory:
    calibfactory = calibname_to_calibfactory[calibname]
    calibfunc = calibfactory(valid_preacts=kaggle_predlogits,
                             valid_labels=kaggle_onehot_labels)
    calibname_to_calibfunc[calibname] = calibfunc
    calibname_to_calibkaggle[calibname] = calibfunc(kaggle_predlogits)
    calibname_to_calibmessidor[calibname] = calibfunc(messidor_predlogits)
    
adaptncalib_pairs = [
    ('bbse-hard', 'None'),
    ('bbse-soft', 'None'),
    ('bbse-soft', 'TS'),
    ('bbse-soft', 'BCTS'),
    #('bbse-soft', 'CVBCTS'),
    ('em', 'None'),
    ('em', 'TS'),
    ('em', 'BCTS'),
    #('em', 'CVBCTS'),
]

adaptncalib_to_preds = OrderedDict()
for (adaptname, calibname) in adaptncalib_pairs:
    calibkaggle_preds = calibname_to_calibkaggle[calibname]
    calibmessidor_preds = calibname_to_calibmessidor[calibname]
    adapter = adaptname_to_adapter[adaptname]
    adapter_func = adapter(valid_labels=kaggle_onehot_labels,
                           tofit_initial_posterior_probs=calibmessidor_preds,
                           valid_posterior_probs=calibkaggle_preds)
    adapted_messidor_preds = adapter_func(calibmessidor_preds)
    adaptncalib_to_preds[adaptname+":"+calibname] = adapted_messidor_preds
    

In [29]:
def get_diseased_v_not_accuracy(preds, onehot_labels):
    return np.mean((np.argmax(preds,axis=-1) > 0.0)
                   ==(1-onehot_labels[:,0]))

def get_frac_compatible_preds_messidor(kaggletype_preds,
                                       messidortype_onehot_labels):
    total_compatible_preds = 0
    argmax_kaggle_preds = np.argmax(kaggletype_preds, axis=-1)
    argmax_messidor_labels = np.argmax(messidortype_onehot_labels, axis=-1)
    for the_kaggle_pred, the_messidor_label in zip(argmax_kaggle_preds,
                                                   argmax_messidor_labels):
        if (the_kaggle_pred==0 and the_messidor_label==0):
            compatible=True
        elif (the_kaggle_pred==1 and the_messidor_label==1):
            compatible=True
        elif ((the_kaggle_pred==1 or the_kaggle_pred==2)
              and the_messidor_label==2):
            compatible=True
        elif ( (the_kaggle_pred >= 2) and the_messidor_label==3 ):
            compatible=True
        else:
            compatible=False
        if (compatible):
            total_compatible_preds += 1
    return total_compatible_preds/len(kaggletype_preds)
  
  
def get_cohens_quadratic_kappa_dropping_class2(kaggletype_preds,
                                               messidortype_onehot_labels):
    filtered_kaggletype_preds =\
      kaggletype_preds[messidortype_onehot_labels[:,2]==0]
    filtered_messidortype_onehot_labels =\
      messidortype_onehot_labels[messidortype_onehot_labels[:,2]==0][:,[0,1,3]]
    kaggle_pred_in_messidor_format = np.concatenate(
      [filtered_kaggletype_preds[:,0:1],
       filtered_kaggletype_preds[:,1:2], 
       (filtered_kaggletype_preds[:,2:3]
        +filtered_kaggletype_preds[:,3:4]
        +filtered_kaggletype_preds[:,4:5])], axis=1)
    return abstention.abstention.weighted_kappa_metric(
            predprobs=kaggle_pred_in_messidor_format,
            true_labels=filtered_messidortype_onehot_labels,
            weights=np.array([[0,1,4], [1,0,1], [4,1,0]]),
            mode='argmax')
    


for calibname in calibname_to_calibmessidor:
    print("Calibname", calibname)
    calibkagglepreds = calibname_to_calibkaggle[calibname]
    calibmessidorpreds = calibname_to_calibmessidor[calibname]
    print("kaggle accuracy for diseased v not",
          get_diseased_v_not_accuracy(preds=calibkagglepreds,
                                      onehot_labels=kaggle_onehot_labels))
    print("messidor accuracy for diseased v not",
          get_diseased_v_not_accuracy(preds=calibmessidorpreds,
                                      onehot_labels=messidor_onehot_labels))
    print("messidor total compatible preds",
          get_frac_compatible_preds_messidor(
              kaggletype_preds=calibmessidorpreds,
              messidortype_onehot_labels=messidor_onehot_labels))
    print("quadratic kappa:", get_cohens_quadratic_kappa_dropping_class2(
            kaggletype_preds=calibmessidorpreds,
            messidortype_onehot_labels=messidor_onehot_labels))

Calibname None
kaggle accuracy for diseased v not 0.8847467273762094
messidor accuracy for diseased v not 0.7991666666666667
messidor total compatible preds 0.7566666666666667
quadratic kappa: 0.8189883216936649
Calibname TS
kaggle accuracy for diseased v not 0.8847467273762094
messidor accuracy for diseased v not 0.7991666666666667
messidor total compatible preds 0.7566666666666667
quadratic kappa: 0.8192805941731682
Calibname BCTS
kaggle accuracy for diseased v not 0.885885031303358
messidor accuracy for diseased v not 0.7766666666666666
messidor total compatible preds 0.7558333333333334
quadratic kappa: 0.8131604698704631


In [30]:
for adaptncalib in adaptncalib_to_preds:
    messidor_preds = adaptncalib_to_preds[adaptncalib]
    print(adaptncalib,"Accuracy for diseased v not",
          get_diseased_v_not_accuracy(preds=messidor_preds,
                                      onehot_labels=messidor_onehot_labels))
    print(adaptncalib,"messidor total compatible preds:",
            get_frac_compatible_preds_messidor(
                kaggletype_preds=messidor_preds,
                messidortype_onehot_labels=messidor_onehot_labels))
    print(adaptncalib, "quadratic kappa:",
          get_cohens_quadratic_kappa_dropping_class2(
              kaggletype_preds=messidor_preds,
              messidortype_onehot_labels=messidor_onehot_labels))

bbse-hard:None Accuracy for diseased v not 0.81
bbse-hard:None messidor total compatible preds: 0.7566666666666667
bbse-hard:None quadratic kappa: 0.8047529769853167
bbse-soft:None Accuracy for diseased v not 0.8108333333333333
bbse-soft:None messidor total compatible preds: 0.7516666666666667
bbse-soft:None quadratic kappa: 0.8207403386433388
bbse-soft:TS Accuracy for diseased v not 0.8133333333333334
bbse-soft:TS messidor total compatible preds: 0.74
bbse-soft:TS quadratic kappa: 0.80614891765428
bbse-soft:BCTS Accuracy for diseased v not 0.8016666666666666
bbse-soft:BCTS messidor total compatible preds: 0.7691666666666667
bbse-soft:BCTS quadratic kappa: 0.8181710202294614
em:None Accuracy for diseased v not 0.8091666666666667
em:None messidor total compatible preds: 0.755
em:None quadratic kappa: 0.8205445011932048
em:TS Accuracy for diseased v not 0.8133333333333334
em:TS messidor total compatible preds: 0.7491666666666666
em:TS quadratic kappa: 0.8134041783752827
em:BCTS Accuracy 